In [1]:
import datetime as dt
import pandas as pd

### LAG/LEAD: calculate deltas from totals

Spark SQL equivalent:

```
select *,
    lag(total, 1, 0) over (partition by url, service order by ts) as prev
from df
```

Spark code:
```
df = sqlContext.read.csv('social_totals.csv', header=True, inferSchema=True)
df.createOrReplaceTempView('df')
df = sqlContext.sql('select *, lag(total, 1, 0) over (partition by url, service order by ts) as prev from df')
df = df.withColumn('delta', df['total'] - df['prev'])
```

In [2]:
df = pd.read_csv('social_totals.csv')
df.head(3)

,url,ts,service,total
0,url1,2018-08-15 00:00:00,tw,1
1,url1,2018-08-15 00:05:00,tw,4
2,url1,2018-08-15 00:11:00,tw,5


In [142]:
# using shift
df = pd.read_csv('social_totals.csv')
df = (df.assign(prev=df.sort_values(['ts'])
                       .groupby(['url', 'service'])
                       .total
                       .shift(1)
                       .fillna(0)
                       .astype(int)))
df = df.assign(delta=df['total']-df['prev'])
df.head(3)

,url,ts,service,total,prev,delta
0,url1,2018-08-15 00:00:00,tw,1,0,1
1,url1,2018-08-15 00:05:00,tw,4,1,3
2,url1,2018-08-15 00:11:00,tw,5,4,1


In [14]:
df = pd.read_csv('social_totals.csv')
df.head(3)

,url,ts,service,total
0,url1,2018-08-15 00:00:00,tw,1
1,url1,2018-08-15 00:05:00,tw,4
2,url1,2018-08-15 00:11:00,tw,5


In [15]:
# using diff
df = df.assign(delta=df.sort_values(['ts'])
                       .groupby(['url', 'service'])
                       .diff(1))
df.loc[df['delta'].isnull(), 'delta'] = df['total']
df.head(10)

,url,ts,service,total,delta
0,url1,2018-08-15 00:00:00,tw,1,1.0
1,url1,2018-08-15 00:05:00,tw,4,3.0
2,url1,2018-08-15 00:11:00,tw,5,1.0
3,url1,2018-08-15 00:18:00,tw,8,3.0
4,url1,2018-08-15 00:21:00,tw,12,4.0
5,url1,2018-08-15 00:30:00,tw,25,13.0
6,url1,2018-08-15 00:35:00,tw,41,16.0
7,url1,2018-08-15 00:38:00,tw,45,4.0
8,url1,2018-08-15 00:41:00,tw,59,14.0
9,url1,2018-08-15 00:00:00,fb,5,5.0


### SUM OVER: calculate totals from deltas

Spark SQL:

```
select *, 
    sum(delta) over (partition by url, service order by ts) as total 
from social_deltas
```

Spark code:

```
df = sqlContext.read.csv('social_deltas.csv', header=True, inferSchema=True)
df.createOrReplaceTempView('df')
df = sqlContext.sql('select *, sum(delta) over (partition by url, service order by ts) as total from df')
```

In [5]:
df = pd.read_csv('social_deltas.csv')
df.head(3)

,url,ts,service,delta
0,url1,2018-08-15 00:00:00,tw,1
1,url1,2018-08-15 00:05:00,tw,3
2,url1,2018-08-15 00:11:00,tw,1


In [6]:
df = pd.read_csv('social_deltas.csv')
df = df.assign(total=df.sort_values(['ts']).groupby(['url', 'service']).delta.cumsum())
df.head(3)

,url,ts,service,delta,total
0,url1,2018-08-15 00:00:00,tw,1,1
1,url1,2018-08-15 00:05:00,tw,3,4
2,url1,2018-08-15 00:11:00,tw,1,5


### RANK, DENSE_RANK, PERCENT_RANK: rank things in group

Spark code:

```
df = sqlContext.read.csv('social_totals_agg.csv', header=True, inferSchema=True)
df.createOrReplaceTempView('df')
df = sqlContext.sql("""
select * from (
   select *,
          rank() over (partition by service order by total desc) as rnk
   from df)
where rnk <= 3
""")
```

to select top 3 urls for each service by number of social shares.

In [113]:
df = pd.read_csv('social_totals_agg.csv')
df = (df.assign(rn=df.groupby(['service'])['total']
                     .rank(method='first', ascending=False))
        .query('rn <= 3')
        .sort_values(['service', 'rn']))
df.head(10)

,url,service,total,rn
12,url3,fb,500,1.0
11,url2,fb,370,2.0
14,url5,fb,300,3.0
9,url10,tw,260,1.0
4,url5,tw,175,2.0
8,url9,tw,150,3.0


### ROWS/RANGE: resize the window

In [106]:
df = pd.read_csv('social_totals.csv', parse_dates=['ts'])
df[(df['service'] == 'fb') & (df['url'] == 'url1')].head(10)

,url,ts,service,total
9,url1,2018-08-15 00:00:00,fb,5
10,url1,2018-08-15 00:05:00,fb,20
11,url1,2018-08-15 00:11:00,fb,31
12,url1,2018-08-15 00:18:00,fb,45
13,url1,2018-08-15 00:21:00,fb,59
14,url1,2018-08-15 00:30:00,fb,67
15,url1,2018-08-15 00:35:00,fb,110
16,url1,2018-08-15 00:38:00,fb,230
17,url1,2018-08-15 00:41:00,fb,360


Average values that fall within range of 5min.

Spark code:
```
df = sqlContext.read.csv('social_totals.csv', header=True, inferSchema=True)
df.createOrReplaceTempView('df')
sqlContext.sql("""
select *, avg(total) over (partition by url, service 
                           order by ts 
                           range between interval 5 minutes preceding and current row) as total_avg5min 
from df
""")
```

In [77]:
df = pd.read_csv('social_totals.csv', parse_dates=['ts'])
ro = (df.groupby(['url', 'service'])
        .rolling('5min', on='ts', min_periods=1)
        .mean()
        .reset_index(drop=True))
df = df.merge(ro, 
              on=['url', 'service', 'ts'], 
              suffixes=('', '_avg5min'))
df.head(10)


,url,ts,service,total,total_avg5min
0,url1,2018-08-15 00:00:00,tw,1,1.0
1,url1,2018-08-15 00:05:00,tw,4,4.0
2,url1,2018-08-15 00:11:00,tw,5,5.0
3,url1,2018-08-15 00:18:00,tw,8,8.0
4,url1,2018-08-15 00:21:00,tw,12,10.0
5,url1,2018-08-15 00:30:00,tw,25,25.0
6,url1,2018-08-15 00:35:00,tw,41,41.0
7,url1,2018-08-15 00:38:00,tw,45,43.0
8,url1,2018-08-15 00:41:00,tw,59,52.0
9,url1,2018-08-15 00:00:00,fb,5,5.0


Each value is a mean of total in the same row + 2 previous values.

Spark code:
```
df = sqlContext.read.csv('social_totals.csv', header=True, inferSchema=True)
df.createOrReplaceTempView('df')
df = sqlContext.sql("""
select *, avg(total) over (partition by url, service 
                           order by ts 
                           rows between 2 preceding and current row) as total_avg3
from df
""")
```

In [75]:
df = pd.read_csv('social_totals.csv', parse_dates=['ts'])
ro = (df.groupby(['url', 'service'])
        .rolling(3, on='ts', min_periods=1)
        .mean()
        .reset_index(drop=True))
df = df.merge(ro, 
              on=['url', 'service', 'ts'], 
              suffixes=('', '_avg3'))
df.head(10)


,url,ts,service,total,total_avg3
0,url1,2018-08-15 00:00:00,tw,1,1.000000
1,url1,2018-08-15 00:05:00,tw,4,2.500000
2,url1,2018-08-15 00:11:00,tw,5,3.333333
3,url1,2018-08-15 00:18:00,tw,8,5.666667
4,url1,2018-08-15 00:21:00,tw,12,8.333333
5,url1,2018-08-15 00:30:00,tw,25,15.000000
6,url1,2018-08-15 00:35:00,tw,41,26.000000
7,url1,2018-08-15 00:38:00,tw,45,37.000000
8,url1,2018-08-15 00:41:00,tw,59,48.333333
9,url1,2018-08-15 00:00:00,fb,5,5.000000
